# L10: Building a Single Agent from Scratch
# L10: 싱글 에이전트를 처음부터 구축하기

이 레슨에서는 L1~L9에서 배운 개념들을 종합하여 **하나의 완성된 에이전트**를 단계별로 만들어갑니다.

프레임워크 설명보다 **실제 구축 과정**에 집중합니다.

## 우리가 만들 에이전트: 연구 도우미 (Research Assistant)

**기능:**
- 웹 검색으로 정보 수집
- 메모 저장 및 조회
- 대화 기억 (세션 유지)
- 실시간 스트리밍 응답
- 구조화된 리포트 출력
- 사용자 권한에 따른 동적 동작
- 민감한 작업 전 사용자 승인

## Setup
## 설정

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env, doublecheck_pkgs

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")  # check environmental variables
doublecheck_pkgs(pyproject_path="pyproject.toml", verbose=True)   # check packages

OPENAI_API_KEY=****kPYA
LANGSMITH_API_KEY=****3193
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials
Python 3.13.9 satisfies requires-python: >=3.11,<3.14
package                | required | installed | status | path                                                                            
---------------------- | -------- | --------- | ------ | --------------------------------------------------------------------------------
langgraph              | >=1.0.0  | 1.0.1     | ✅ OK   | …t_projects/lca-langchainV1-essentials/python/.venv/lib/python3.13/site-packages
langchain              | >=1.0.0  | 1.0.2     | ✅ OK   | …t_projects/lca-langchainV1-essentials/python/.venv/lib/python3.13/site-packages
langchain-core         | >=1.0.0  | 1.0.1     | ✅ OK   | …t_projects/lca-langchainV1-essentials/python/.venv/lib/python3.13/site-packages
langchain-openai       | >=1.0.0  | 1.0.1     | ✅ OK   | …t_projects/lca-langchainV1-essentials/python/.venv/lib/python3.13/site-packages
langchain-anthrop

---
## Step 1: 기본 에이전트 생성

가장 단순한 형태의 에이전트부터 시작합니다.

In [2]:
from langchain.agents import create_agent

# 가장 기본적인 에이전트: 모델 + 시스템 프롬프트
agent_v1 = create_agent(
    model="openai:gpt-4o-mini",
    system_prompt="You are a research assistant. Help users find and organize information."
)

In [3]:
# 테스트
result = agent_v1.invoke({"messages": "AI 에이전트의 핵심 구성요소가 뭐야?"})
print(result["messages"][-1].content)

AI 에이전트의 핵심 구성요소는 다음과 같습니다:

1. **퍼셉션 (Perception)**: AI 에이전트는 주변 환경에서 정보를 수집하고 이해하는 능력이 필요합니다. 이는 센서, 데이터 입력 또는 다른 정보원을 통해 이루어질 수 있습니다.

2. **추론 (Reasoning)**: 수집한 데이터를 기반으로 의사결정을 내리는 과정입니다. 에이전트는 알고리즘과 모델을 사용하여 정보를 분석하고 적절한 행동을 결정합니다.

3. **행동 (Action)**: 최종적으로 AI 에이전트는 의사결정에 따라 실제 행동을 취해야 합니다. 이는 로봇 팔을 움직이거나, 사용자에게 정보를 제공하거나, 시스템의 상태를 변경하는 등 다양할 수 있습니다.

4. **학습 (Learning)**: AI 에이전트는 경험을 통해 성능을 향상시킬 수 있는 능력이 있어야 합니다. 머신러닝 알고리즘을 통해 과거의 데이터를 분석하고 패턴을 학습합니다.

5. **기억 (Memory)**: 경험을 저장하고 필요할 때 접근할 수 있는 기능입니다. 이는 에이전트가 이전의 경험과 지식을 활용하여 더 나은 결정을 내릴 수 있게 합니다.

6. **목표 설정 (Goal Setting)**: 에이전트는 특정 목표를 가지고 이를 달성하기 위해 행동해야 합니다. 목표 설정은 에이전트의 행동 방향성을 결정합니다.

이러한 구성요소들은 AI 에이전트가 주어진 작업을 수행하고 환경에 적응하며 발전하는 데 중요한 역할을 합니다.


**문제점:** 에이전트가 자체 지식에만 의존합니다. 실시간 정보나 외부 데이터에 접근할 수 없습니다.

---
## Step 2: 도구 추가

에이전트가 실제 작업을 수행할 수 있도록 도구를 추가합니다.

In [15]:
from langchain_core.tools import tool
from datetime import datetime

# 메모 저장소 (간단한 인메모리 구현)
notes_db = {}

@tool
def save_note(title: str, content: str) -> str:
    """Save a research note with a title and content."""
    notes_db[title] = {
        "content": content,
        "created_at": datetime.now().isoformat()
    }
    return f"Note '{title}' saved successfully."

@tool
def get_note(title: str) -> str:
    """Retrieve a saved note by its title."""
    if title in notes_db:
        note = notes_db[title]
        return f"Title: {title}\nContent: {note['content']}\nCreated: {note['created_at']}"
    return f"Note '{title}' not found."

@tool
def list_notes() -> str:
    """List all saved note titles."""
    if not notes_db:
        return "No notes saved yet."
    return "Saved notes: " + ", ".join(notes_db.keys())

@tool
def web_search(query: str) -> str:
    """Search the web for information. Returns simulated search results."""
    # 실제 구현에서는 검색 API 연동
    return f"[Simulated search results for '{query}']: Found relevant information about {query}. Key points: 1) Definition and overview, 2) Recent developments, 3) Practical applications."

In [16]:
SYSTEM_PROMPT_V2 = """You are a research assistant with the following capabilities:
- Search the web for information
- Save important findings as notes
- Retrieve and list saved notes

When researching a topic:
1. Search for relevant information
2. Summarize key findings
3. Offer to save important points as notes
"""

agent_v2 = create_agent(
    model="openai:gpt-4o-mini",
    tools=[web_search, save_note, get_note, list_notes],
    system_prompt=SYSTEM_PROMPT_V2,
)

In [17]:
# 테스트: 검색하고 메모 저장
for step in agent_v2.stream(
    {"messages": "LLM Agent에 대해 검색하고 핵심 내용을 'LLM_Agent_Basics'라는 제목으로 저장해줘"},
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

LLM Agent에 대해 검색하고 핵심 내용을 'LLM_Agent_Basics'라는 제목으로 저장해줘


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  web_search (call_uaJWMzhNOB2ntmZDK4TM6wWy)
 Call ID: call_uaJWMzhNOB2ntmZDK4TM6wWy
  Args:
    query: LLM Agent
================================= Tool Message =================================
Name: web_search

[Simulated search results for 'LLM Agent']: Found relevant information about LLM Agent. Key points: 1) Definition and overview, 2) Recent developments, 3) Practical applications.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================
Tool Calls:
  save_note (call_FR8bp6LaDIWT9QZPz3CIw37b)
 Call ID: call_FR8bp6LaDIWT9QZPz3CIw37b
  Args:
    title: LLM_Agent_Basics
    content: 1) Definition: LLM (Large Language Model) Agent refers to AI models that can interact, respond, and perform tasks based on natural language understanding. 2) Recent developments: Advances in training methodologies and architecture have enhanced LLM capabilities, making them more responsive and context-aware. 3) Practical applications: LLM Agents are used in customer service automation, content generation, virtual assistants, and more, improving workflow efficiencies.
================================= Tool Message =================================
Name: save_note

Note 'LLM_Agent_Basics' saved successfully.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

I found key information about LLM Agents and saved it under the title **LLM_Agent_Basics**. Here are the main points:

1. **Definition**: LLM (Large Language Model) Agent refers to AI models that can interact, respond, and perform tasks based on natural language understanding.
2. **Recent developments**: Advances in training methodologies and architecture have enhanced LLM capabilities, making them more responsive and context-aware.
3. **Practical applications**: LLM Agents are used in customer service automation, content generation, virtual assistants, and more, improving workflow efficiencies.

If you need anything else or further details, feel free to ask!


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [18]:
# 저장된 메모 확인
result = agent_v2.invoke({"messages": "저장된 메모 목록 보여줘"})
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


저장된 메모 목록은 다음과 같습니다:

- LLM_Agent_Basics

더 궁금하신 점이나 이 메모에 대해 보고 싶으신 내용이 있나요?


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


**문제점:** 각 호출이 독립적입니다. 이전 대화 내용을 기억하지 못합니다.

---
## Step 3: 메모리 추가

대화 히스토리를 유지하여 맥락을 이해하는 에이전트로 발전시킵니다.

In [19]:
from langgraph.checkpoint.memory import InMemorySaver

agent_v3 = create_agent(
    model="openai:gpt-4o-mini",
    tools=[web_search, save_note, get_note, list_notes],
    system_prompt=SYSTEM_PROMPT_V2,
    checkpointer=InMemorySaver(),  # 메모리 추가
)

In [20]:
# thread_id로 세션 구분
config = {"configurable": {"thread_id": "session-1"}}

# 첫 번째 메시지
result = agent_v3.invoke(
    {"messages": "내 이름은 민수야. AI 에이전트에 대해 연구하고 있어."},
    config
)
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


안녕하세요, 민수님! AI 에이전트에 대해 연구하고 계시군요. 어떤 특정한 주제나 질문이 있으신가요? 관련된 정보를 검색해 드릴 수 있습니다.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [21]:
# 두 번째 메시지 - 이전 대화 기억하는지 확인
result = agent_v3.invoke(
    {"messages": "내 이름이 뭐라고 했지? 그리고 뭘 연구한다고 했어?"},
    config  # 같은 thread_id 사용
)
print(result["messages"][-1].content)

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


민수님, 당신의 이름은 "민수"이고, AI 에이전트에 대해 연구하고 있다고 하셨습니다. 추가적으로 궁금한 점이나 필요한 정보가 있으시면 말씀해 주세요!


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [22]:
# 다른 세션은 독립적
config_new = {"configurable": {"thread_id": "session-2"}}

result = agent_v3.invoke(
    {"messages": "내 이름이 뭐야?"},
    config_new
)
print(result["messages"][-1].content)  # 모름

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 이름을 알려주시면 그에 맞게 대화할 수 있습니다!


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


**성과:** 이제 에이전트가 대화 맥락을 기억합니다!

---
## Step 4: 실시간 스트리밍

긴 응답도 토큰 단위로 즉시 표시되도록 합니다.

In [23]:
# stream_mode="values": 각 단계의 전체 상태
config = {"configurable": {"thread_id": "stream-test"}}

print("=== stream_mode='values' ===")
for step in agent_v3.stream(
    {"messages": "AI 에이전트의 장점을 3가지만 짧게 말해줘"},
    config,
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

=== stream_mode='values' ===
================================ Human Message =================================

AI 에이전트의 장점을 3가지만 짧게 말해줘


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


================================== Ai Message ==================================

AI 에이전트의 장점은 다음과 같습니다:

1. **효율성 향상**: AI 에이전트는 반복적인 작업을 자동화하여 업무의 효율성을 높이고 시간을 절약할 수 있습니다.

2. **24/7 가용성**: AI 에이전트는 언제든지 작동할 수 있어, 고객 서비스나 지원을 24시간 제공할 수 있습니다.

3. **데이터 분석 능력**: 대량의 데이터를 신속하게 분석하고 패턴을 식별하여 의사결정에 기여할 수 있습니다.


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


In [24]:
# stream_mode="messages": 토큰 단위 스트리밍
config = {"configurable": {"thread_id": "stream-test-2"}}

print("=== stream_mode='messages' (토큰 단위) ===")
for token, metadata in agent_v3.stream(
    {"messages": "AI 에이전트의 장점을 3가지만 짧게 말해줘"},
    config,
    stream_mode="messages"
):
    print(token.content, end="", flush=True)
print()  # 줄바꿈

=== stream_mode='messages' (토큰 단위) ===
AI 에이전트의 장점

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


은 다음과 같습니다:

1. **효율성**: 반복적이고 시간이 많이 소요되는 작업을 자동화하여 작업의 속도를 높이고 인적 자원 절약.
2. **정밀도**: 데이터 처리 및 분석에서 높은 정확도를 제공하여 오차를 줄이고 더 나은 결정을 지원.
3. **24/7 가용성**: 언제든지 작동할 수 있어 사용자가 필요할 때 상시 지원을 제공. 

추가적으로 더 알고 싶으시면 말씀해 주세요!


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


---
## Step 5: 구조화된 출력

에이전트가 정해진 형식으로 결과를 반환하도록 합니다.

In [25]:
from pydantic import BaseModel
from typing import List

class ResearchReport(BaseModel):
    """연구 결과 리포트 스키마"""
    topic: str
    summary: str
    key_points: List[str]
    sources: List[str]
    confidence_level: str  # "high", "medium", "low"

In [26]:
# 구조화된 출력을 위한 에이전트
agent_structured = create_agent(
    model="openai:gpt-4o-mini",
    tools=[web_search],
    system_prompt="You are a research assistant. Search for information and compile a structured report.",
    response_format=ResearchReport,
)

In [27]:
result = agent_structured.invoke(
    {"messages": "LangChain에 대해 조사해줘"}
)

# 구조화된 응답
report = result["structured_response"]
print(f"Topic: {report.topic}")
print(f"Summary: {report.summary}")
print(f"Key Points:")
for i, point in enumerate(report.key_points, 1):
    print(f"  {i}. {point}")
print(f"Confidence: {report.confidence_level}")

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTP

Topic: LangChain
Summary: LangChain is a flexible framework designed for building applications using language models. It enables developers to create complex applications that can interact with language models, integrating various tools and features for a robust user experience.
Key Points:
  1. Definition and overview: A framework for language model applications.
  2. Recent developments: Continuous updates and enhancements are being released.
  3. Practical applications: Used in chatbots, automated content generation, and more.
Confidence: High


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


---
## Step 6: 동적 프롬프트

런타임 상황에 따라 에이전트 동작을 변경합니다.

In [ ]:
from dataclasses import dataclass

@dataclass
class UserContext:
    """사용자 컨텍스트"""
    user_name: str
    is_premium: bool
    language: str  # "ko" or "en"

In [ ]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt

PROMPT_TEMPLATE = """You are a research assistant for {user_name}.

{tier_message}

Language preference: Respond in {language}.

Available tools: web_search, save_note, get_note, list_notes
"""

@dynamic_prompt
def personalized_prompt(request: ModelRequest) -> str:
    ctx = request.runtime.context
    
    if ctx.is_premium:
        tier_message = "Premium user: Provide detailed, comprehensive responses with extra insights."
    else:
        tier_message = "Free user: Provide concise responses. Limit to 3 search queries per topic."
    
    language = "Korean" if ctx.language == "ko" else "English"
    
    return PROMPT_TEMPLATE.format(
        user_name=ctx.user_name,
        tier_message=tier_message,
        language=language
    )

In [ ]:
agent_v6 = create_agent(
    model="openai:gpt-4o-mini",
    tools=[web_search, save_note, get_note, list_notes],
    middleware=[personalized_prompt],
    context_schema=UserContext,
    checkpointer=InMemorySaver(),
)

In [ ]:
# 프리미엄 사용자
premium_ctx = UserContext(user_name="김철수", is_premium=True, language="ko")

result = agent_v6.invoke(
    {"messages": "AI 에이전트에 대해 알려줘"},
    context=premium_ctx
)
print("=== Premium User Response ===")
print(result["messages"][-1].content[:500] + "...")

In [ ]:
# 무료 사용자
free_ctx = UserContext(user_name="John", is_premium=False, language="en")

result = agent_v6.invoke(
    {"messages": "Tell me about AI agents"},
    context=free_ctx
)
print("=== Free User Response ===")
print(result["messages"][-1].content)

---
## Step 7: Human-in-the-Loop (HITL)

민감한 작업 전에 사용자 승인을 받습니다.

In [ ]:
# 민감한 작업을 포함한 도구
@tool
def delete_note(title: str) -> str:
    """Delete a saved note. This action cannot be undone."""
    if title in notes_db:
        del notes_db[title]
        return f"Note '{title}' has been deleted."
    return f"Note '{title}' not found."

@tool
def export_all_notes() -> str:
    """Export all notes to external storage. Requires approval."""
    return f"Exported {len(notes_db)} notes to external storage."

In [ ]:
from langchain.agents.middleware import HumanInTheLoopMiddleware

agent_v7 = create_agent(
    model="openai:gpt-4o-mini",
    tools=[web_search, save_note, get_note, list_notes, delete_note, export_all_notes],
    system_prompt="You are a research assistant. Help users manage their research notes.",
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            # delete와 export는 승인 필요
            interrupt_on={
                "delete_note": {"allowed_decisions": ["approve", "reject"]},
                "export_all_notes": {"allowed_decisions": ["approve", "reject"]},
            }
        )
    ],
)

In [ ]:
# 테스트를 위해 메모 추가
notes_db["test_note"] = {"content": "This is a test note", "created_at": "2024-01-01"}
print(f"Current notes: {list(notes_db.keys())}")

In [ ]:
from langgraph.types import Command

config = {"configurable": {"thread_id": "hitl-test"}}

# 삭제 요청
result = agent_v7.invoke(
    {"messages": "test_note 메모를 삭제해줘"},
    config=config
)

# 인터럽트 확인
if "__interrupt__" in result:
    interrupt_info = result['__interrupt__'][-1].value['action_requests'][-1]
    print("🛑 승인 필요!")
    print(f"   도구: {interrupt_info['tool_name']}")
    print(f"   인자: {interrupt_info['tool_input']}")
    
    # 사용자 승인 (실제로는 UI에서 입력받음)
    user_decision = "approve"  # or "reject"
    
    if user_decision == "approve":
        result = agent_v7.invoke(
            Command(resume={"decisions": [{"type": "approve"}]}),
            config=config
        )
        print("\n✅ 승인됨")
    else:
        result = agent_v7.invoke(
            Command(resume={"decisions": [{"type": "reject", "message": "User cancelled"}]}),
            config=config
        )
        print("\n❌ 거부됨")

print(f"\n결과: {result['messages'][-1].content}")
print(f"현재 메모: {list(notes_db.keys())}")

---
## 최종: 모든 기능을 통합한 완성 에이전트

지금까지 배운 모든 것을 하나의 에이전트에 통합합니다.

In [ ]:
from dataclasses import dataclass
from typing import List, Optional
from pydantic import BaseModel

@dataclass
class AppContext:
    """애플리케이션 컨텍스트"""
    user_id: str
    user_name: str
    is_premium: bool
    language: str

In [ ]:
from langchain_core.tools import tool
from langgraph.config import get_stream_writer
from datetime import datetime

# 사용자별 메모 저장소
user_notes = {}

@tool
def search_web(query: str) -> str:
    """Search the web for information on a topic."""
    writer = get_stream_writer()
    writer(f"🔍 Searching for: {query}")
    # 실제로는 검색 API 호출
    writer(f"✅ Found results for: {query}")
    return f"Search results for '{query}': [Relevant information found]"

@tool
def save_research_note(title: str, content: str, tags: str) -> str:
    """Save a research note with title, content, and comma-separated tags."""
    from langgraph.runtime import get_runtime
    runtime = get_runtime(AppContext)
    user_id = runtime.context.user_id
    
    if user_id not in user_notes:
        user_notes[user_id] = {}
    
    user_notes[user_id][title] = {
        "content": content,
        "tags": [t.strip() for t in tags.split(",")],
        "created_at": datetime.now().isoformat()
    }
    return f"Note '{title}' saved with tags: {tags}"

@tool
def get_my_notes() -> str:
    """Get all notes for the current user."""
    from langgraph.runtime import get_runtime
    runtime = get_runtime(AppContext)
    user_id = runtime.context.user_id
    
    if user_id not in user_notes or not user_notes[user_id]:
        return "No notes found."
    
    result = []
    for title, note in user_notes[user_id].items():
        result.append(f"- {title} (tags: {', '.join(note['tags'])})")
    return "Your notes:\n" + "\n".join(result)

@tool
def delete_research_note(title: str) -> str:
    """Delete a research note. Requires user approval."""
    from langgraph.runtime import get_runtime
    runtime = get_runtime(AppContext)
    user_id = runtime.context.user_id
    
    if user_id in user_notes and title in user_notes[user_id]:
        del user_notes[user_id][title]
        return f"Note '{title}' deleted."
    return f"Note '{title}' not found."

In [ ]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt

FINAL_PROMPT = """You are a research assistant for {user_name}.

User tier: {tier}
{tier_instructions}

Language: Always respond in {language}.

Capabilities:
- search_web: Search for information
- save_research_note: Save notes with tags
- get_my_notes: List user's saved notes
- delete_research_note: Delete a note (requires approval)

Be helpful, concise, and organize information clearly.
"""

@dynamic_prompt
def build_prompt(request: ModelRequest) -> str:
    ctx = request.runtime.context
    
    tier = "Premium" if ctx.is_premium else "Free"
    tier_instructions = (
        "Provide comprehensive, detailed responses with insights."
        if ctx.is_premium else
        "Provide concise responses. Suggest premium for advanced features."
    )
    language = "Korean" if ctx.language == "ko" else "English"
    
    return FINAL_PROMPT.format(
        user_name=ctx.user_name,
        tier=tier,
        tier_instructions=tier_instructions,
        language=language
    )

In [ ]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

# 최종 에이전트: 모든 기능 통합
final_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[search_web, save_research_note, get_my_notes, delete_research_note],
    middleware=[
        build_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "delete_research_note": {"allowed_decisions": ["approve", "reject"]}
            }
        )
    ],
    context_schema=AppContext,
    checkpointer=InMemorySaver(),
)

print("✅ Final Research Assistant Agent created!")

In [ ]:
# 데모: 완성된 에이전트 테스트
user_ctx = AppContext(
    user_id="user-001",
    user_name="민수",
    is_premium=True,
    language="ko"
)
config = {"configurable": {"thread_id": "final-demo"}}

print("=" * 60)
print("🤖 Research Assistant Demo")
print("=" * 60)

In [ ]:
# 1. 검색 + 스트리밍
print("\n[1] 검색 및 메모 저장")
for chunk in final_agent.stream(
    {"messages": "LangChain에 대해 검색하고 핵심 내용을 메모로 저장해줘. 태그는 'AI, Framework, LLM'으로 해줘"},
    config,
    context=user_ctx,
    stream_mode=["values", "custom"]
):
    if chunk[0] == "custom":
        print(f"  {chunk[1]}")
    elif chunk[0] == "values":
        msg = chunk[1]["messages"][-1]
        if hasattr(msg, 'content') and msg.content:
            pass  # 최종 응답은 마지막에 출력

# 최종 응답
result = final_agent.invoke({"messages": "방금 뭘 했어?"}, config, context=user_ctx)
print(f"\n응답: {result['messages'][-1].content}")

In [ ]:
# 2. 메모리 확인 (이전 대화 기억)
print("\n[2] 메모리 확인")
result = final_agent.invoke(
    {"messages": "내 이름이 뭐라고 했지? 저장된 메모도 보여줘"},
    config,
    context=user_ctx
)
print(result["messages"][-1].content)

---
## 요약: 싱글 에이전트 구축 체크리스트

| 단계 | 구성요소 | 코드 |  |
|------|----------|------|---|
| 1 | 기본 에이전트 | `create_agent(model, system_prompt)` | L1 |
| 2 | 도구 | `@tool` decorator + `tools=[...]` | L4 |
| 3 | 메모리 | `checkpointer=InMemorySaver()` | L6 |
| 4 | 스트리밍 | `stream_mode="messages"` or `"values"` | L3 |
| 5 | 구조화 출력 | `response_format=YourSchema` | L7 |
| 6 | 동적 프롬프트 | `@dynamic_prompt` + `middleware=[...]` | L8 |
| 7 | HITL | `HumanInTheLoopMiddleware` | L9 |

### 핵심 패턴

```python
agent = create_agent(
    model="openai:gpt-4o-mini",           # LLM
    tools=[tool1, tool2],                  # 도구들
    system_prompt="...",                   # 또는 middleware로 동적 프롬프트
    middleware=[dynamic_prompt, hitl],     # 미들웨어
    context_schema=MyContext,              # 런타임 컨텍스트
    checkpointer=InMemorySaver(),          # 메모리
    response_format=MySchema,              # 구조화 출력 (선택)
)
```

---
## 다음 단계

이 에이전트를 더 발전시키려면:

1. **실제 검색 API 연동**: Tavily, Serper, Google Search API
2. **영구 저장소**: PostgreSQL, Redis로 checkpointer 교체
3. **MCP 도구 추가**: `langchain-mcp-adapters`로 외부 MCP 서버 연결 (L5)
4. **멀티 에이전트**: 여러 에이전트 협업 (다음 코스)
5. **프로덕션 배포**: LangGraph Cloud, API 서버화